# Simple audio recognition: Recognizing keywords

Bu çalışma dosyasında, WAV biçimindeki ses dosyalarına nasıl ön işlem uygulanacağını ve temel bir <a href="https://en.wikipedia.org/wiki/Speech_recognition" class="external">otomatik konuşma tanıma (automatic speech recognition)</a> (ASR) modeli oluşturulup eğitileceğini gösterir. 

On farklı kelimeyi tanımak için **"aşağı (down)", "git (go)", "sol (left)", "hayır (no)", "sağ (right)", "dur (stop)", "yukarı (up)" ve "evet (yes)"** gibi komutların kısa (bir saniye veya daha az) ses kliplerini içeren [Speech Commands dataset](https://www.tensorflow.org/datasets/catalog/speech_commands) bir bölümünü kullanacaksınız (<a href="https://arxiv.org/abs/1804.03209" class="external">Warden, 2018</a>)

Gerçek dünyadaki konuşma ve ses tanıma <a href="https://ai.googleblog.com/search/label/Speech%20Recognition" class="external">sistemleri</a> karmaşıktır. Ancak, MNIST veri kümesiyle görüntü sınıflandırması gibi, bu eğitim size ilgili teknikler hakkında temel bir anlayış vermelidir.

## Kurulum

Gerekli modülleri ve bağımlılıkları import edin. Bu çalışma dosyasında görselleştirme için <a href="https://seaborn.pydata.org/" class="external">seaborn</a> kullanacağınızı unutmayın.

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Mini Konuşma Komutları Veri Kümesini yükleyin

Veri yüklemeyle zaman kazanmak için Konuşma Komutları veri kümesinin daha küçük bir sürümüyle çalışacaksınız. [Orijinal veri kümesi](https://www.tensorflow.org/datasets/catalog/speech_commands), adresindeki 105.000'den fazla ses dosyasından oluşur. <a href="https://www.aelius.com/njh/wavemetatools/doc/riffmci.pdf" class="external">WAV (Dalga formu) ses dosyası formatı</a>, 35 farklı kelime söyleyen kişiler.

Daha küçük Konuşma Komutları veri kümelerini içeren `mini_speech_commands.zip dosyasını `tf.keras.utils.get_file` ile indirin ve çıkarın:

In [ ]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

Veri kümesinin ses klipleri, her konuşma komutuna karşılık gelen sekiz klasörde saklanır: "hayır", "evet", "aşağı", "git", "sol", "yukarı", "sağ" ve "dur":

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != 'README.md']
print('Commands:', commands)

Ses kliplerini `filenames` adlı bir listeye çıkarın ve karıştırın:

In [ ]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print('Number of total examples:', num_samples)
print('Number of examples per label:',
      len(tf.io.gfile.listdir(str(data_dir/commands[0]))))
print('Example file tensor:', filenames[0])

`filenames` sırasıyla 80:10:10 oranını kullanarak eğitim, doğrulama ve test kümelerine ayırın:

In [ ]:
train_files = filenames[:6400]
val_files = filenames[6400: 6400 + 800]
test_files = filenames[-800:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

## Ses dosyalarını ve etiketlerini okuyun

Bu bölümde, dalga formları ve ilgili etiketler için kodu çözülmüş tensörler oluşturarak veri setini önceden işleyeceksiniz.

- Her WAV dosyası, saniyede belirli sayıda örnek içeren zaman serisi verileri içerir.

- Her örnek, belirli bir zamanda ses sinyalinin <a href="https://en.wikipedia.org/wiki/Amplitude" class="external">genliğini (amplitude)</a> temsil eder.

- Bir <a href="https://en.wikipedia.org/wiki/Audio_bit_depth" class="external">16-bit</a> sistemde, mini Konuşma Komutları veri kümesindeki WAV dosyaları gibi, genlik değerler -32.768 ile 32.767 arasındadır.

- Bu veri kümesi için <a href="https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Audio_sampling" class="external">örnek hızı (sample rate)</a> 16kHz'dir.

`tf.audio.decode_wav`tarafından döndürülen tensörün boyutu `[samples, channels]` şeklindedir; burada `channels` mono için `1` veya stereo için `2`dir. 
Mini Konuşma Komutları veri kümesi yalnızca mono kayıtları içerir.

In [ ]:
test_file = tf.io.read_file(DATASET_PATH+'/down/0a9f9af7_nohash_0.wav')
test_audio, _ = tf.audio.decode_wav(contents=test_file)
test_audio.shape

In [ ]:
display.display(display.Audio(DATASET_PATH+'/down/0a9f9af7_nohash_0.wav', rate=16000))

Şimdi, veri kümesinin ham WAV ses dosyalarını ses tensörlerine önişleyen bir işlev tanımlayalım:

In [ ]:
def decode_audio(audio_binary):
  # Decode WAV-encoded audio files to `float32` tensors, normalized
  # to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
  audio, _ = tf.audio.decode_wav(contents=audio_binary)
  # Since all the data is single channel (mono), drop the `channels`
  # axis from the array.
  return tf.squeeze(audio, axis=-1)

Her dosya için üst dizinleri kullanarak etiketler oluşturan bir fonksiyon tanımlayın:

- Dosya yollarını `tf.RaggedTensor`lara ayırın (düzensiz boyutlara sahip tensörler - farklı uzunluklara sahip olabilen dilimler).

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
  # Note: You'll use indexing here instead of tuple unpacking to enable this
  # to work in a TensorFlow graph.
  return parts[-2]

Hepsini bir araya getiren başka bir yardımcı fonksiyon (`get_waveform_and_label`) tanımlayın:

- Giriş, WAV ses dosyası adıdır.
- Çıktı, denetimli öğrenmeye hazır ses ve etiket tensörlerini içeren bir demettir.

In [ ]:
def get_waveform_and_label(file_path):
  label = get_label(file_path)
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, label

Ses etiketi çiftlerini çıkarmak için eğitim setini oluşturun:

- Daha önce tanımlanan `get_waveform_and_label` kullanarak `Dataset.from_tensor_slices` ve `Dataset.map` ile bir `tf.data.Dataset` oluşturun.

Daha sonra benzer bir prosedür kullanarak doğrulama ve test setlerini oluşturacaksınız.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

files_ds = tf.data.Dataset.from_tensor_slices(train_files)

waveform_ds = files_ds.map(
    map_func=get_waveform_and_label,
    num_parallel_calls=AUTOTUNE)

Birkaç ses dalga biçimi çizelim:

In [ ]:
rows = 3
cols = 3
n = rows * cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 12))

for i, (audio, label) in enumerate(waveform_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  ax.plot(audio.numpy())
  ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
  label = label.numpy().decode('utf-8')
  ax.set_title(label)

plt.show()

## Dalga formlarını spektrogramlara dönüştürün

Veri kümesindeki dalga biçimleri zaman alanında temsil edilir. Ardından, dalga biçimlerini zaman içinde frekans değişikliklerini gösteren ve dalga biçimlerini spektrogramlara dönüştürmek için <a href="https://en.wikipedia.org/wiki/Short-time_Fourier_transform" class="external">short-time Fourier transform (STFT)</a> hesaplayarak dalga biçimlerini zaman etki alanı sinyallerinden zaman-frekans etki alanı sinyallerine dönüştüreceksiniz. 2D görüntüler olarak temsil edilir. Modeli eğitmek için <a href="https://en.wikipedia.org/wiki/Spectrogram" clas="external">spektrogram</a> görüntülerini sinir ağınıza besleyeceksiniz.

Fourier dönüşümü (`tf.signal.fft`), bir sinyali bileşen frekanslarına dönüştürür, ancak tüm zaman bilgilerini kaybeder. Buna karşılık, STFT (`tf.signal.stft`) sinyali zaman pencerelerine böler ve her pencerede bir Fourier dönüşümü çalıştırarak zaman bilgisini korur ve üzerinde standart evrişimler çalıştırabileceğiniz bir 2B tensör döndürür.

Dalga biçimlerini spektrogramlara dönüştürmek için bir yardımcı program işlevi oluşturun:

- Dalga biçimlerinin aynı uzunlukta olması gerekir, böylece onları spektrogramlara dönüştürdüğünüzde sonuçlar benzer boyutlara sahip olur. Bu, bir saniyeden daha kısa olan ses kliplerinin sıfırlanmasıyla (`tf.zeros` kullanılarak) yapılabilir.

- `tf.signal.stft`yi çağırırken, oluşturulan spektrogram "görüntü" neredeyse kare olacak şekilde `frame_length` ve `frame_step` parametrelerini seçin. STFT parametreleri seçimi hakkında daha fazla bilgi için, ses sinyali işleme ve STFT ile ilgili bu <a href="https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe" class="external">this Coursera video</a> videosuna bakın.

- STFT, büyüklük ve fazı temsil eden bir dizi karmaşık sayı üretir. Ancak, bu öğreticide yalnızca `tf.signal.stft` çıktısına `tf.abs` uygulayarak türetebileceğiniz büyüklüğü kullanacaksınız.

In [ ]:
def get_spectrogram(waveform):
  # Zero-padding for an audio waveform with less than 16,000 samples.
  input_len = 16000
  waveform = waveform[:input_len]
  zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
  # Cast the waveform tensors' dtype to float32.
  waveform = tf.cast(waveform, dtype=tf.float32)
  # Concatenate the waveform with `zero_padding`, which ensures all audio
  # clips are of the same length.
  equal_length = tf.concat([waveform, zero_padding], 0)
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

Ardından, verileri keşfetmeye başlayın. Bir örneğin gerginleştirilmiş dalga formunun ve ilgili spektrogramın şekillerini yazdırın ve orijinal sesi çalın:

In [ ]:
for waveform, label in waveform_ds.take(1):
  label = label.numpy().decode('utf-8')
  spectrogram = get_spectrogram(waveform)

print('Label:', label)
print('Waveform shape:', waveform.shape)
print('Spectrogram shape:', spectrogram.shape)
print('Audio playback')
display.display(display.Audio(waveform, rate=16000))

Şimdi, bir spektrogramı görüntülemek için bir fonksiyon tanımlayın:

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

Örneğin zaman içindeki dalga biçimini ve karşılık gelen spektrogramı (zaman içindeki frekanslar) çizin:

In [ ]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])

plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.show()

Şimdi, dalga biçimi veri kümesini spektrogramlara ve bunlara karşılık gelen etiketleri tamsayı kimlikleri olarak dönüştüren bir işlev tanımlayın:

In [ ]:
def get_spectrogram_and_label_id(audio, label):
  spectrogram = get_spectrogram(audio)
  label_id = tf.argmax(label == commands)
  return spectrogram, label_id


`get_spectrogram_and_label_id` öğesini `Dataset.map` ile veri kümesinin öğeleri arasında eşleyin:

In [ ]:
spectrogram_ds = waveform_ds.map(
  map_func=get_spectrogram_and_label_id,
  num_parallel_calls=AUTOTUNE)

Veri kümesinin farklı örnekleri için spektrogramları inceleyin:

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

for i, (spectrogram, label_id) in enumerate(spectrogram_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  plot_spectrogram(spectrogram.numpy(), ax)
  ax.set_title(commands[label_id.numpy()])
  ax.axis('off')

plt.show()

## Modeli oluşturun ve eğitin

Doğrulama ve test kümelerinde, eğitim kümesindeki ön işlemesini tekrarlayın:

In [ ]:
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)

  output_ds = files_ds.map(
      map_func=get_waveform_and_label,
      num_parallel_calls=AUTOTUNE)
  
  output_ds = output_ds.map(
      map_func=get_spectrogram_and_label_id,
      num_parallel_calls=AUTOTUNE)
  
  return output_ds

In [ ]:
train_ds = spectrogram_ds
val_ds = preprocess_dataset(val_files)
test_ds = preprocess_dataset(test_files)

Model eğitimi için eğitim ve doğrulama kümelerini toplu olarak oluşturun:

In [ ]:
batch_size = 64
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

Modeli eğitirken okuma gecikmesini azaltmak için `Dataset.cache` ve `Dataset.prefetch` işlemlerini ekleyin:

In [ ]:
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

Ses dosyalarını spektrogram görüntülerine dönüştürdüğünüz için model için basit bir evrişimsel sinir ağı (CNN) kullanacaksınız.

"`tf.keras.Sequential` modeliniz aşağıdaki Keras ön işleme katmanlarını kullanacaktır:

- `tf.keras.layers.Resizing`: modelin daha hızlı çalışmasını sağlamak için girdiyi aşağı örneklemek (downsample) için.

- `tf.keras.layers.Normalization`: görüntüdeki her pikseli ortalama ve standart sapmaya göre normalleştirmek için.

`Normalization` katmanı için, toplu istatistikleri (yani, ortalama ve standart sapmayı) hesaplamak için ilk önce eğitim verilerinde `adapt` yönteminin çağrılması gerekir.

In [ ]:
for spectrogram, _ in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)
num_labels = len(commands)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

Keras modelini Adam optimizer ve çapraz entropi kaybıyla (cross-entropy loss) yapılandırın:

In [ ]:
#from_logits 
#y_pred öğesinin bir logits tensörü olmasının beklenip beklenmediği. 
#Varsayılan olarak, y_pred'in bir olasılık dağılımını kodladığını varsayıyoruz.

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

Modeli 10 tur(epoch) fazla eğitin:

In [ ]:
EPOCHS = 11
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Modelinizin eğitim sırasında nasıl geliştiğini kontrol etmek için eğitim ve doğrulama kaybı eğrilerini çizelim:

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

## Model performansını değerlendirin

Modeli test kümesinde çalıştırın ve modelin performansını kontrol edin:

In [ ]:
test_audio = []
test_labels = []

for audio, label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [ ]:
y_pred = np.argmax(model.predict(test_audio), axis=1)
y_true = test_labels

test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')

### Karışıklık matrisi (confusion matrix) 

Modelin her birini ne kadar iyi sınıflandırdığını kontrol etmek için bir <a href="https://developers.google.com/machine-learning/glossary#confusion-matrix" class="external">karışıklık matrisi (confusion matrix)</a> kullanın. 

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))

sns.heatmap(confusion_mtx,
            xticklabels=commands,
            yticklabels=commands,
            annot=True, fmt='g')

plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

## Bir ses dosyasında çıkarımı çalıştır

Son olarak, **"hayır"** diyen birinin giriş ses dosyasını kullanarak modelin tahmin çıktısını doğrulayın. 
Modeliniz ne kadar iyi performans gösteriyor?

In [ ]:
sample_file = data_dir/'no/01bb6a2a_nohash_0.wav'

sample_ds = preprocess_dataset([str(sample_file)])

for spectrogram, label in sample_ds.batch(1):
  prediction = model(spectrogram)
  plt.bar(commands, tf.nn.softmax(prediction[0]))
  plt.title(f'Predictions for "{commands[label[0]]}"')
  plt.show()

In [ ]:
prediction

In [ ]:
tf.nn.softmax(prediction[0])

In [ ]:
display.display(display.Audio(str(data_dir/'no/01bb6a2a_nohash_0.wav'), rate=16000))

Çıktının önerdiği gibi, modelinizin sesli komutu "hayır" olarak tanıması gerekir.

## Sonraki adımlar

Bu öğretici, TensorFlow ve Python ile evrişimli bir sinir ağı kullanarak basit ses sınıflandırmasının/otomatik konuşma tanımanın nasıl gerçekleştirileceğini gösterdi. Daha fazla bilgi edinmek için aşağıdaki kaynakları göz önünde bulundurun:

- The [Sound classification with YAMNet](https://www.tensorflow.org/hub/tutorials/yamnet) tutorial shows how to use transfer learning for audio classification.
- The notebooks from <a href="https://www.kaggle.com/c/tensorflow-speech-recognition-challenge/overview" class="external">Kaggle's TensorFlow speech recognition challenge</a>.
- The 
<a href="https://codelabs.developers.google.com/codelabs/tensorflowjs-audio-codelab/index.html#0" class="external">TensorFlow.js - Audio recognition using transfer learning codelab</a> teaches how to build your own interactive web app for audio classification.
- <a href="https://arxiv.org/abs/1709.04396" class="external">A tutorial on deep learning for music information retrieval</a> (Choi et al., 2017) on arXiv.
- TensorFlow also has additional support for [audio data preparation and augmentation](https://www.tensorflow.org/io/tutorials/audio) to help with your own audio-based projects.
- Consider using the <a href="https://librosa.org/" class="external">librosa</a> library—a Python package for music and audio analysis.